In [1]:
import retro
from gym import Env
# imort the space shapes for the environment
from gym.spaces import MultiBinary, box
# import numnpy to calculate frame delta 
import numpy as np
# import open cv for grayscaling
import cv2
import time 
# this use to slow the movement or make it normal playback import time

In [ ]:
retro.data.list_games()

In [3]:
env = retro.make(game='StreetFighterIISpecialChampionEdition-Genesis')

In [5]:
env.close() # closes the environment important given we can only run one at a time 


In [ ]:
env.action_space # multi-binary spaces take a combination of binary flags in order to  make up an action in this case multibinary(12)
 # means a combination of 12 binary flags 

In [ ]:
env.action_space.sample() # sample the observation space

In [ ]:
len([1,0,1,0,1,1,1,1,1,0,1,1,])

In [ ]:
env.observation_space

In [ ]:
#Game loop
# reset game to starting state
# if we die means the done flase the obs or env.reset the game will reset again 
obs = env.reset()
# set flag to false telling us if it dies or not 
done = False 
# 
for game in range(1):
    while not done:
        if done:
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample()) # this randomly taking action inside of our game 
        time.sleep(0.01)# use to slow the process or make it watchable the gameplay
        print(reward)
    
    obs = env.reset()
    done = False

      

In [ ]:
info


Diet frame before and after
-Observation preprocess - grayscale, frame delta, resize the frame so we have less pixels
-Filter the a ction - parameter
-Reward function  - set this to score

In [17]:
MultiBinary(12).sample()

array([0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1], dtype=int8)

In [18]:
from matplotlib import pyplot as plt

In [ ]:
gray = cv2.cvtColor(obs, cv2.COLOR_BGR2GRAY)# we can use RGB or GRAY we get blue image bc opencv reference in bgr and matplotlib reference in rgb

In [ ]:
resize = cv2.resize(gray,(84,84), interpolation=cv2.INTER_CUBIC)# this control pixel 
channels = np.reshape(resize, (84,84,1))

In [ ]:
plt.imshow(cv2.cvtColor(channels, cv2.COLOR_BGR2RGB))

In [ ]:
#what our code does
# 1. frame 
#2. preprocess 200x256x3 to 84x84x1
#3. change in pixels current_frame - the last frame (previous_frame)

Your code seems mostly correct, but there are a few issues that might be causing the error:

box is not defined. It should be gym.spaces.Box.
In the prerocess method, you’re using obs but it’s not defined in the method’s scope. It should be observation.
In the prerocess method, you’re calling resize as if it were a function. It should be resized.
In the step method, you’re calling self.preprocess(obs), but the method is named prerocess.
bellow this is the error and decumented code 


In [ ]:
# #setup environment 

# # passing ENV from the import from the top 
# class Streetfighter(Env):
#     def __init__(self):
#         super().__init__()
#         # specify action space and observation space
#         self.observation_space = box(low = 0, high = 255, shape=(84,84, 1), dtype= np.uint8)# this are the image size i think (200, 256, 3) and the 255 high and low ois the color
#         self.action_space = MultiBinary(12) # random move
#         #startup and instance of the game
#         self.game =  retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_action=retro.Actions.FILTERED)# this filtered our action inside of our env rather than having our most binary 
#     def reset(self):
#         #return first frame
#         obs = self.game.reset() # this going to let us return to our observation space below also the retunr obs
#         obs = self.preprocess(obs)
#         self.previous_frame = obs
#         #create a attribute to hold the score delta
#         self.score = 0 
#         return obs
#     def prerocess(self, observation ): # this is optional
#         gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)# we can use RGB or GRAY we get blue image bc opencv reference in bgr and matplotlib reference in rgb
#         # resize 
#         resize = cv2.resize(gray,(84,84), interpolation=cv2.INTER_CUBIC)# we cant use 84,84,1 here beacuse of intercubic so we use np reshape in var channel
#         # add channels value
#         channels = np.reshape(resize(84,84,1)) # since we're reshaping our pix we need to chnage the shape of the observatopm s[ac varoan;e ]
#         return channels
#     def step(self, action):
#         # take a step
#         obs, reward, done, info = self.game.step(action) # whatever our game up there variable game we take step
#         obs = self.preprocess(obs)# this takes data on top u see the observation or obs top

#         # getting frame delt
#         #obs - previous_frame {formula}
#         frame_delta = obs - self.previous_frame
#         self.previous_frame = obs  

#         #reshape the reward function 
#         reward = info ['score'] - self.score# subtract the current score to self score in top which is 0
#         # resetting the current score 
#         self.score = info['score']

#         return frame_delta, reward, done, info
#     def render(self, *args, **kwargs): # part of env setter we need to unpack this thats why we have args and kwargs
#         self.game.render() # and passing this
     
#     def close(self):
#         self.game.close
        
    

This are well refined code below


In [38]:
from gym import Env
from gym.spaces import Box, MultiBinary
import retro
import cv2
import numpy as np

class Streetfighter(Env):
    def __init__(self):
        super().__init__()
        # specify action space and observation space
        self.observation_space = Box(low = 0, high = 255, shape=(84,84, 1), dtype= np.uint8)
        self.action_space = MultiBinary(12) 
        self.game =  retro.make(game='StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions=retro.Actions.FILTERED)
    def reset(self):
        obs = self.game.reset() 
        obs = self.preprocess(obs)
        self.previous_frame = obs
        self.score = 0 
        return obs
    def preprocess(self, observation ): 
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray,(84,84), interpolation=cv2.INTER_CUBIC)
        channels = np.reshape(resized,(84,84,1)) 
        return channels
    def step(self, action): ##The step method is called with an action that the agent has decided to take.
        obs, reward, done, info = self.game.step(action) ##The self.game.step(action) line executes the action in the game, and returns the new observation, the reward for the action, whether the game is done, and some additional info.
        obs = self.preprocess(obs)##The new observation is preprocessed and stored in obs.
        frame_delta = obs - self.previous_frame # calculated the previous obs and current obs 
        self.previous_frame = obs  #The previous observation is updated to the new one. which is the = obs
        reward = info ['score'] - self.score # The reward is then recalculated as the difference between the current score (info['score']) and the score at the previous step (self.score).
        self.score = info['score'] # and stored to the new one which is updated the updated is the current score
        return frame_delta, reward, done, info #Finally, the frame_delta, recalculated reward, done, and info are returned.
    def render(self, *args, **kwargs): 
        self.game.render() 
    def close(self):
        self.game.close()


In [ ]:
info 

In [39]:
env = Streetfighter()

In [40]:
env.observation_space.shape

(84, 84, 1)

In [42]:
env.action_space.shape

(12,)

In [ ]:
#Game loop
# reset game to starting state
# if we die means the done flase the obs or env.reset the game will reset again 
obs = env.reset()
# set flag to false telling us if it dies or not 
done = False 
# 
for game in range(1):
    while not done:
        if done:
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample()) # this randomly taking action inside of our game 
        time.sleep(0.01)# use to slow the process or make it watchable the gameplay
        if reward > 0:
            print(reward)
       
    
    obs = env.reset()
    done = False

      